In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.decomposition import PCA
from scipy.cluster.hierarchy import dendrogram, linkage
from matplotlib.backends.backend_pdf import PdfPages
from kneed import KneeLocator



file_path = r"C:\Users\anon\Downloads\adult\adult.data"
column_names = [
    "age", "workclass", "fnlwgt", "education", "education-num", 
    "marital-status", "occupation", "relationship", "race", "sex", 
    "capital-gain", "capital-loss", "hours-per-week", "native-country", "income"
]

adult = pd.read_csv(file_path, names=column_names, sep=", ", engine="python")
print(adult.head())


cat_cols = adult.select_dtypes(include="object").columns
for col in cat_cols:
    adult[col] = LabelEncoder().fit_transform(adult[col])

scaler = StandardScaler()
adult_scaled = scaler.fit_transform(adult)


sse = []
k_range = range(1, 11)
for k in k_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(adult_scaled)
    sse.append(kmeans.inertia_)

knee = KneeLocator(k_range, sse, curve='convex', direction='decreasing')
optimal_k = knee.knee
if optimal_k is None:
    optimal_k = 4  # якщо не знайшов "ліктя", задаємо 4
print(f"Оптимальна кількість кластерів (k) за методом ліктя: {optimal_k}")

kmeans = KMeans(n_clusters=optimal_k, random_state=42, n_init=10)
adult['kmeans_cluster'] = kmeans.fit_predict(adult_scaled)

centers = pd.DataFrame(kmeans.cluster_centers_, columns=adult.columns[:-1])
centers['cluster_size'] = pd.Series(adult['kmeans_cluster']).value_counts().sort_index()
print("\nЦентри кластерів та кількість елементів (KMeans):")
print(centers)

pca = PCA(n_components=2)
adult_pca = pca.fit_transform(adult_scaled)


sample_size = 3000  # підвибірка для ієрархічного кластерування
sample_idx = np.random.choice(adult_scaled.shape[0], sample_size, replace=False)
adult_sample = adult_scaled[sample_idx]

linked = linkage(adult_sample, method='ward')
agg = AgglomerativeClustering(n_clusters=optimal_k)
agg_labels = agg.fit_predict(adult_sample)

agg_counts = pd.Series(agg_labels).value_counts().sort_index()
print("\nКількість елементів по кластерам (Agglomerative, підвибірка):")
print(agg_counts)


comparison = pd.DataFrame({
    'KMeans_sample': pd.Series(adult['kmeans_cluster'].values[sample_idx]).value_counts().sort_index(),
    'Agglomerative': agg_counts
})
print("\nПорівняння кластерів (підвибірка):")
print(comparison)


pdf_path = r"C:\Users\anon\Downloads\adult\Adult_Clustering_Report.pdf"
pdf = PdfPages(pdf_path)

# Лінія SSE
plt.figure(figsize=(8,5))
plt.plot(k_range, sse, 'o-', color='blue')
plt.xlabel('Кількість кластерів')
plt.ylabel('SSE')
plt.title('Метод ліктя (KMeans)')
plt.grid(True)
pdf.savefig(); plt.close()

# PCA-кластери
plt.figure(figsize=(8,6))
for cluster in range(optimal_k):
    plt.scatter(adult_pca[adult['kmeans_cluster']==cluster,0],
                adult_pca[adult['kmeans_cluster']==cluster,1],
                label=f'Cluster {cluster}', s=30)
plt.scatter(pca.transform(kmeans.cluster_centers_)[:,0],
            pca.transform(kmeans.cluster_centers_)[:,1],
            color='black', marker='X', s=200, label='Centers')
plt.title('KMeans кластери (PCA)')
plt.legend()
pdf.savefig(); plt.close()

# Дендрограма (підвибірка)
plt.figure(figsize=(12,6))
dendrogram(linked, truncate_mode='level', p=5)
plt.title('Дендрограма (Ward, підвибірка)')
plt.xlabel('Об’єкти')
plt.ylabel('Відстань')
pdf.savefig(); plt.close()

# Таблиці
for df, title in [(centers.round(2), 'Центри кластерів (KMeans)'),
                  (comparison, 'Порівняння KMeans vs Agglomerative (підвибірка)')]:
    fig, ax = plt.subplots(figsize=(12,2))
    ax.axis('tight'); ax.axis('off')
    ax.table(cellText=df.values, colLabels=df.columns, loc='center')
    plt.title(title)
    pdf.savefig(); plt.close()

pdf.close()
print(f"PDF-звіт збережено: {pdf_path}")


   age         workclass  fnlwgt  education  education-num  \
0   39         State-gov   77516  Bachelors             13   
1   50  Self-emp-not-inc   83311  Bachelors             13   
2   38           Private  215646    HS-grad              9   
3   53           Private  234721       11th              7   
4   28           Private  338409  Bachelors             13   

       marital-status         occupation   relationship   race     sex  \
0       Never-married       Adm-clerical  Not-in-family  White    Male   
1  Married-civ-spouse    Exec-managerial        Husband  White    Male   
2            Divorced  Handlers-cleaners  Not-in-family  White    Male   
3  Married-civ-spouse  Handlers-cleaners        Husband  Black    Male   
4  Married-civ-spouse     Prof-specialty           Wife  Black  Female   

   capital-gain  capital-loss  hours-per-week native-country income  
0          2174             0              40  United-States  <=50K  
1             0             0             